In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

In [3]:
data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder' 



In [4]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted(os.listdir(root_dir))
        self.labels = [self.extract_label(img) for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code, just commented out for testing purposes
        # label = img_name
        return label

# Example usage:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5), (0.5))]) 

custom_dataset = CustomImageDataset(root_dir=data_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))
# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25003)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

15800
11850
3950


In [5]:
#hyper parameters
num_epochs = 30
batch_size = 1
learning_rate = 0.0005

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test = DataLoader(test_set, batch_size=batch_size, shuffle=False)

class ConvNet(nn.Module): # note need to find out image size
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,8,10, padding='same') #in_channels, out_channels, kernel_size
        self.normalise1 = nn.BatchNorm2d(8)
        # self.pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
        self.pool1 = nn.AvgPool2d(10, stride=10)
        self.conv2 = nn.Conv2d(8, 16, 10, padding='same')
        self.normalise2 = nn.BatchNorm2d(16)
        self.pool2 = nn.AvgPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(16, 32, 10, padding='same')
        self.normalise3 = nn.BatchNorm2d(32) 
        self.conv4 = nn.Conv2d(32, 32, 10, padding='same')
        # self.fc1 = nn.Linear(16*3*3, 120) # 3x3 is the size of the image after 2 conv layers, 16 is the number of channels, 120 is the number of nodes in the hidden layer
        # self.fc2 = nn.Linear(120,84)
        # self.fc3 = nn.Linear(60, 1)
        self.fc = nn.Linear(32*5*5, 1)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        x = self.pool1(F.relu(self.normalise1(self.conv1(x)))) 
        x = self.pool2(F.relu(self.normalise2(self.conv2(x)))) 
        x = F.relu(self.normalise3(self.conv3(x)))
        x = F.relu(self.normalise3(self.conv4(x)))
        x = x.view(-1, 32*5*5)  #flatten
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc(x)
        return x

model = ConvNet().to(device)

# loss and optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train):
        images = images.to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        print(labels)
        labels = labels.float() #this is the right code, just commented out for testing purposes
        # labels = float(labels[0][-17:-5])
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            

print("Finished Training")

c:\Users\cs2036\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1009.)
  return F.conv2d(input, weight, bias, self.stride,
c:\Users\cs2036\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([8.2175], dtype=torch.float64)
epoch 1/30, step 1/11850, loss = 65.1061
tensor([7.8236], dtype=torch.float64)
epoch 1/30, step 2/11850, loss = 26.5139
tensor([14.0570], dtype=torch.float64)
epoch 1/30, step 3/11850, loss = 79.2027
tensor([2.9723], dtype=torch.float64)
epoch 1/30, step 4/11850, loss = 12.7546
tensor([25.1087], dtype=torch.float64)
epoch 1/30, step 5/11850, loss = 350.0385
tensor([14.5373], dtype=torch.float64)
epoch 1/30, step 6/11850, loss = 52.6283
tensor([11.7508], dtype=torch.float64)
epoch 1/30, step 7/11850, loss = 17.8941
tensor([17.8033], dtype=torch.float64)
epoch 1/30, step 8/11850, loss = 94.1519
tensor([17.5512], dtype=torch.float64)
epoch 1/30, step 9/11850, loss = 82.7802
tensor([19.5075], dtype=torch.float64)
epoch 1/30, step 10/11850, loss = 118.7206
tensor([2.2368], dtype=torch.float64)
epoch 1/30, step 11/11850, loss = 46.0682
tensor([13.7409], dtype=torch.float64)
epoch 1/30, step 12/11850, loss = 19.1537
tensor([12.4303], dtype=torch.float64)


In [36]:
with torch.no_grad(): # no need to calculate gradient
    squared_difference = 0
    for images, labels in test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        #value, index
        _, predictions = torch.max(outputs, 1)
        squared_difference += (predictions - labels) ** 2
    
    rmse = torch.sqrt(squared_difference / len(test))
    print(f'RMSE = {rmse}')

RMSE = tensor([14.5283], dtype=torch.float64)


In [ ]:
import time
from tqdm import tqdm

count =0
for i in tqdm(range(100)):
    count += 1
    time.sleep(1)
print(count)